# INSTALLING THE NECESSARY LIBRARIES

In [1]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install matplotlib
!pip install scipy
!pip install pickle-mixin
!pip install seaborn
!pip install Flask
!pip install Xgboost

  Preparing metadata (setup.py) ... done
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=5991 sha256=7bedd9321524bf556842f12d176d29734476c285be8a9a5e6e78e8f05f473864
  Stored in directory: /root/.cache/pip/wheels/3e/c6/e9/d1b0a34e1efc6c3ec9c086623972c6de6317faddb2af0a619c
Successfully built pickle-mixin


# IMPORTING THE NECESSARY LIBRARIES

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle as pkl
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,RidgeClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,StandardScaler

# IMPORTING THE DATASET AND READING IT

In [19]:
from google.colab import files
uploaded = files.upload()

# Assuming the file is named 'data.xlsx'
dt = pd.read_excel('data.xlsx')


Saving data.xlsx to data.xlsx


In [20]:
dt.head()

,S.NO,Age,Gender,Place(location where the patient lives),Duration of alcohol consumption(years),Quantity of alcohol consumption (quarters/day),Type of alcohol consumed,Hepatitis B infection,Hepatitis C infection,Diabetes Result,...,Direct (mg/dl),Indirect (mg/dl),Total Protein (g/dl),Albumin (g/dl),Globulin (g/dl),AL.Phosphatase (U/L),SGOT/AST (U/L),SGPT/ALT (U/L),USG Abdomen (diffuse liver or not),Predicted Value(Out Come-Patient suffering from liver cirrosis or not)
0,1,55,male,rural,12,2,branded liquor,negative,negative,YES,...,4.0,3.0,6.0,3.0,4.0,150.0,56,34,YES,YES
1,2,55,male,rural,12,2,branded liquor,negative,negative,YES,...,4.0,3.0,6.0,3.0,4.0,150.0,56,34,YES,YES
2,3,55,male,rural,12,2,branded liquor,negative,negative,YES,...,4.0,3.0,6.0,3.0,4.0,150.0,56,34,YES,YES
3,4,55,male,rural,12,2,branded liquor,negative,negative,NO,...,4.0,3.0,6.0,3.0,4.0,150.0,56,34,YES,YES
4,5,55,female,rural,12,2,branded liquor,negative,negative,YES,...,4.0,3.0,6.0,3.0,4.0,150.0,56,34,YES,YES


# SEPERATING DEPENDANT AND INDEPENDANT VARIABLE

In [21]:
x = dt.iloc[:, :-1]
y = dt.iloc[:, -1]


# HANDLING MISSING NUMERIC DATA

In [22]:
numeric_columns = ['TCH', 'TG', 'LDL', 'HDL', 'PCV  (%)', 'RBC  (million cells/microliter)', 'MCV   (femtoliters/cell)',
                   'MCH  (picograms/cell)', 'MCHC  (grams/deciliter)', 'Total Count', 'Lymphocytes  (%)',
                   'Monocytes   (%)', 'Eosinophils   (%)', 'Basophils  (%)', 'Platelet Count  (lakhs/mm)',
                   'Indirect     (mg/dl)', 'Total Protein     (g/dl)', 'Albumin   (g/dl)', 'Globulin  (g/dl)',
                   'AL.Phosphatase      (U/L)']
numeric_imputer = SimpleImputer(strategy='mean')
x[numeric_columns] = numeric_imputer.fit_transform(x[numeric_columns])


# ENCODING CATEGORICAL DATA

In [23]:
bp_split = dt['Blood pressure (mmhg)'].str.split('/', expand=True)
x['Systolic'] = pd.to_numeric(bp_split[0], errors='coerce')
x['Diastolic'] = pd.to_numeric(bp_split[1], errors='coerce')
x = x.drop(columns=['Blood pressure (mmhg)'])

In [24]:
cat_columns = ['Gender', 'Place(location where the patient lives)', 'Type of alcohol consumed',
               'Hepatitis B infection', 'Hepatitis C infection', 'Diabetes Result', 'Obesity',
               'Family history of cirrhosis/ hereditary', 'USG Abdomen (diffuse liver or  not)']
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), cat_columns)], remainder='passthrough')
x_encoded = ct.fit_transform(x)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# PREPROCESSED DATA

In [25]:
preprocessed_data = pd.DataFrame(x_encoded, columns=ct.get_feature_names_out())
preprocessed_data['Predicted Value'] = y_encoded

In [26]:
preprocessed_data.isnull().sum()

encoder__Gender_female                                       0
encoder__Gender_female                                       0
encoder__Gender_male                                         0
encoder__Gender_transgender                                  0
encoder__Place(location where the patient lives)_ ocun       0
encoder__Place(location where the patient lives)_rural       0
encoder__Place(location where the patient lives)_urban       0
encoder__Place(location where the patient lives)_nan         0
encoder__Type of alcohol consumed_ branded liquor            0
encoder__Type of alcohol consumed_both                       0
encoder__Type of alcohol consumed_branded liquor             0
encoder__Type of alcohol consumed_country liquor             0
encoder__Hepatitis B infection_Positive                      0
encoder__Hepatitis B infection_negative                      0
encoder__Hepatitis B infection_positive                      0
encoder__Hepatitis C infection_Positive                

# FEATURE SCALING

In [27]:
scaler = StandardScaler()
normalized_features = scaler.fit_transform(preprocessed_data.drop(columns=['Predicted Value']))
normalized_data = pd.DataFrame(normalized_features, columns=preprocessed_data.columns[:-1])
normalized_data['Predicted Value'] = preprocessed_data['Predicted Value']
print(normalized_data.head())

   encoder__Gender_female  encoder__Gender_female   encoder__Gender_male  \
0               -0.508211                -0.108234              0.534342   
1               -0.508211                -0.108234              0.534342   
2               -0.508211                -0.108234              0.534342   
3               -0.508211                -0.108234              0.534342   
4                1.967688                -0.108234             -1.871462   

   encoder__Gender_transgender  \
0                    -0.072739   
1                    -0.072739   
2                    -0.072739   
3                    -0.072739   
4                    -0.072739   

   encoder__Place(location where the patient lives)_ ocun  \
0                                          -0.032461        
1                                          -0.032461        
2                                          -0.032461        
3                                          -0.032461        
4                                

# SPLITTING DATA INTO TRAINING AND TEST SETS

In [28]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(normalized_data.iloc[:, :-1], normalized_data['Predicted Value'], test_size=0.2, random_state=42)

print("Shape of x_train:", x_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of x_train: (760, 58)
Shape of x_test: (190, 58)
Shape of y_train: (760,)
Shape of y_test: (190,)


In [29]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

RandomForestClassifier()

In [30]:
x_train

,encoder__Gender_female,encoder__Gender_female,encoder__Gender_male,encoder__Gender_transgender,encoder__Place(location where the patient lives)_ ocun,encoder__Place(location where the patient lives)_rural,encoder__Place(location where the patient lives)_urban,encoder__Place(location where the patient lives)_nan,encoder__Type of alcohol consumed_ branded liquor,encoder__Type of alcohol consumed_both,...,remainder__Direct (mg/dl),remainder__Indirect (mg/dl),remainder__Total Protein (g/dl),remainder__Albumin (g/dl),remainder__Globulin (g/dl),remainder__AL.Phosphatase (U/L),remainder__SGOT/AST (U/L),remainder__SGPT/ALT (U/L),remainder__Systolic,remainder__Diastolic
936,-0.508211,9.239244,-1.871462,-0.072739,-0.032461,-0.954709,1.274484,-0.405235,-0.032461,-0.659580,...,-1.103319,-4.312503e-01,-2.284780e-01,0.641342,0.193160,-0.824978,0.907197,0.269563,-0.162504,0.043365
290,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,-0.954709,1.274484,-0.405235,-0.032461,-0.659580,...,-0.014781,5.112869e-01,3.224009e-01,0.092745,0.571994,-0.824978,-0.765337,-0.848990,0.464542,-0.669376
544,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,-0.954709,1.274484,-0.405235,-0.032461,1.516117,...,-0.740473,-1.373787e+00,-1.251539e+00,-0.364420,-0.943341,-1.191237,0.392572,1.420074,-0.162504,-0.669376
275,1.967688,-0.108234,-1.871462,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,1.516117,...,1.291464,-6.197577e-01,6.371888e-01,0.047028,0.344693,0.420301,-0.893993,-0.657238,1.091588,1.468845
109,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,-0.659580,...,0.710911,-4.312503e-01,1.109371e+00,0.092745,0.571994,-0.458720,-0.089890,0.716984,-0.162504,0.471009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,-0.659580,...,1.400318,1.359570e+00,-4.645689e-01,-0.364420,-0.185674,-0.824978,0.135259,0.557190,0.464542,-0.241732
270,1.967688,-0.108234,-1.871462,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,1.516117,...,1.291464,-6.197577e-01,6.371888e-01,0.047028,0.344693,0.420301,-0.893993,-0.657238,1.091588,1.468845
860,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,1.516117,...,-0.123635,-5.859988e-15,4.892787e-15,-0.364420,0.950827,0.896438,0.199587,-0.177858,-0.789550,6.458027
435,1.967688,-0.108234,-1.871462,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,-0.659580,...,-0.559050,-9.025189e-01,-4.645689e-01,-0.135838,-0.564507,0.713308,2.901373,3.785013,-3.297734,-1.382116


In [31]:
y_train

936    1
290    0
544    0
275    0
109    0
      ..
106    0
270    0
860    0
435    0
102    2
Name: Predicted Value, Length: 760, dtype: int64

# LOGISTIC REGRESSION

In [33]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=500)
lr.fit(x_train,y_train)

LogisticRegression(max_iter=500)

In [34]:
x_train

,encoder__Gender_female,encoder__Gender_female,encoder__Gender_male,encoder__Gender_transgender,encoder__Place(location where the patient lives)_ ocun,encoder__Place(location where the patient lives)_rural,encoder__Place(location where the patient lives)_urban,encoder__Place(location where the patient lives)_nan,encoder__Type of alcohol consumed_ branded liquor,encoder__Type of alcohol consumed_both,...,remainder__Direct (mg/dl),remainder__Indirect (mg/dl),remainder__Total Protein (g/dl),remainder__Albumin (g/dl),remainder__Globulin (g/dl),remainder__AL.Phosphatase (U/L),remainder__SGOT/AST (U/L),remainder__SGPT/ALT (U/L),remainder__Systolic,remainder__Diastolic
936,-0.508211,9.239244,-1.871462,-0.072739,-0.032461,-0.954709,1.274484,-0.405235,-0.032461,-0.659580,...,-1.103319,-4.312503e-01,-2.284780e-01,0.641342,0.193160,-0.824978,0.907197,0.269563,-0.162504,0.043365
290,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,-0.954709,1.274484,-0.405235,-0.032461,-0.659580,...,-0.014781,5.112869e-01,3.224009e-01,0.092745,0.571994,-0.824978,-0.765337,-0.848990,0.464542,-0.669376
544,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,-0.954709,1.274484,-0.405235,-0.032461,1.516117,...,-0.740473,-1.373787e+00,-1.251539e+00,-0.364420,-0.943341,-1.191237,0.392572,1.420074,-0.162504,-0.669376
275,1.967688,-0.108234,-1.871462,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,1.516117,...,1.291464,-6.197577e-01,6.371888e-01,0.047028,0.344693,0.420301,-0.893993,-0.657238,1.091588,1.468845
109,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,-0.659580,...,0.710911,-4.312503e-01,1.109371e+00,0.092745,0.571994,-0.458720,-0.089890,0.716984,-0.162504,0.471009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,-0.659580,...,1.400318,1.359570e+00,-4.645689e-01,-0.364420,-0.185674,-0.824978,0.135259,0.557190,0.464542,-0.241732
270,1.967688,-0.108234,-1.871462,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,1.516117,...,1.291464,-6.197577e-01,6.371888e-01,0.047028,0.344693,0.420301,-0.893993,-0.657238,1.091588,1.468845
860,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,1.516117,...,-0.123635,-5.859988e-15,4.892787e-15,-0.364420,0.950827,0.896438,0.199587,-0.177858,-0.789550,6.458027
435,1.967688,-0.108234,-1.871462,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,-0.659580,...,-0.559050,-9.025189e-01,-4.645689e-01,-0.135838,-0.564507,0.713308,2.901373,3.785013,-3.297734,-1.382116


In [35]:
y_train

936    1
290    0
544    0
275    0
109    0
      ..
106    0
270    0
860    0
435    0
102    2
Name: Predicted Value, Length: 760, dtype: int64

# KNN

In [36]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(x_train,y_train)

KNeighborsClassifier()

In [37]:
x_train

,encoder__Gender_female,encoder__Gender_female,encoder__Gender_male,encoder__Gender_transgender,encoder__Place(location where the patient lives)_ ocun,encoder__Place(location where the patient lives)_rural,encoder__Place(location where the patient lives)_urban,encoder__Place(location where the patient lives)_nan,encoder__Type of alcohol consumed_ branded liquor,encoder__Type of alcohol consumed_both,...,remainder__Direct (mg/dl),remainder__Indirect (mg/dl),remainder__Total Protein (g/dl),remainder__Albumin (g/dl),remainder__Globulin (g/dl),remainder__AL.Phosphatase (U/L),remainder__SGOT/AST (U/L),remainder__SGPT/ALT (U/L),remainder__Systolic,remainder__Diastolic
936,-0.508211,9.239244,-1.871462,-0.072739,-0.032461,-0.954709,1.274484,-0.405235,-0.032461,-0.659580,...,-1.103319,-4.312503e-01,-2.284780e-01,0.641342,0.193160,-0.824978,0.907197,0.269563,-0.162504,0.043365
290,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,-0.954709,1.274484,-0.405235,-0.032461,-0.659580,...,-0.014781,5.112869e-01,3.224009e-01,0.092745,0.571994,-0.824978,-0.765337,-0.848990,0.464542,-0.669376
544,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,-0.954709,1.274484,-0.405235,-0.032461,1.516117,...,-0.740473,-1.373787e+00,-1.251539e+00,-0.364420,-0.943341,-1.191237,0.392572,1.420074,-0.162504,-0.669376
275,1.967688,-0.108234,-1.871462,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,1.516117,...,1.291464,-6.197577e-01,6.371888e-01,0.047028,0.344693,0.420301,-0.893993,-0.657238,1.091588,1.468845
109,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,-0.659580,...,0.710911,-4.312503e-01,1.109371e+00,0.092745,0.571994,-0.458720,-0.089890,0.716984,-0.162504,0.471009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,-0.659580,...,1.400318,1.359570e+00,-4.645689e-01,-0.364420,-0.185674,-0.824978,0.135259,0.557190,0.464542,-0.241732
270,1.967688,-0.108234,-1.871462,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,1.516117,...,1.291464,-6.197577e-01,6.371888e-01,0.047028,0.344693,0.420301,-0.893993,-0.657238,1.091588,1.468845
860,-0.508211,-0.108234,0.534342,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,1.516117,...,-0.123635,-5.859988e-15,4.892787e-15,-0.364420,0.950827,0.896438,0.199587,-0.177858,-0.789550,6.458027
435,1.967688,-0.108234,-1.871462,-0.072739,-0.032461,1.047440,-0.784631,-0.405235,-0.032461,-0.659580,...,-0.559050,-9.025189e-01,-4.645689e-01,-0.135838,-0.564507,0.713308,2.901373,3.785013,-3.297734,-1.382116


In [38]:
y_train

936    1
290    0
544    0
275    0
109    0
      ..
106    0
270    0
860    0
435    0
102    2
Name: Predicted Value, Length: 760, dtype: int64

# Hyper Parameter

In [39]:
 from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [40]:
k = np.random.randint(1,50,60)

In [42]:
params = {'n_neighbors': k}

In [44]:
random_search = RandomizedSearchCV(knn, params,n_iter=5,cv=5, n_jobs=-1, verbose=0)
random_search.fit(x_train,y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5, n_jobs=-1,
                   param_distributions={'n_neighbors': array([31, 47,  4,  2, 18,  2, 15, 38, 41, 43,  4,  3, 44, 42, 49, 21, 13,
       14, 38, 30, 10, 12, 34, 29, 30, 18,  9,  1,  5,  7,  3, 44,  2,  1,
        5, 41, 25, 29,  6, 11, 40, 11, 22, 24,  2, 45, 23, 12,  8, 16, 18,
       15, 13, 49, 24, 34,  1, 16, 31, 14])})

In [47]:
print('train_score '+ str(random_search.score(x_train,y_train)))
print('test_score '+ str(random_search.score(x_test,y_test)))

train_score 0.9355263157894737
test_score 0.9368421052631579
